In [1]:
import pandas as pd 
from string import ascii_uppercase as alphabet
import pickle
from bs4 import BeautifulSoup
import requests

# DATA TABULATION

In [ ]:
import pandas as pd
import string

# Define alphabet
alphabet = list(string.ascii_uppercase)

# Read tables from the Wikipedia page
table = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
dict_table = {}

# Ensure the range does not exceed the number of tables available
for letter, i in zip(alphabet, range(11, min(67, len(table)), 7)):
    df = table[i]
    df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
    
    # Check if 'Qualification' column exists before trying to pop it
    if 'Qualification' in df.columns:
        df.pop('Qualification')
    
    dict_table[f'Group {letter}'] = df



In [24]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [25]:
dict_table['Group H']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Brazil,3,2,0,1,3,1,+2,6
1,2,Switzerland,3,2,0,1,4,3,+1,6
2,3,Cameroon,3,1,1,1,4,4,0,4
3,4,Serbia,3,0,1,2,5,8,−3,1


In [26]:
with open('dict_table', 'wb') as output:
    pickle.dump(dict_table, output)

# DATA SCRAPING

In [78]:
#DATA SCRAPING
import requests
import pandas as pd
from bs4 import BeautifulSoup

def get_matches(year):
    years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
             1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
             2018]
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    resp = requests.get(web)
    content = resp.text
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_='footballbox')
    
    home = []
    score = []
    away = []
    
    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())
        
    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    
    # Correctly assign the year to the DataFrame
    df_football['year'] = year
    
    return df_football

# Ensure the years list is defined outside the function
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018,]

# Fetch matches for each year
fifa = [get_matches(year) for year in years]

# Concatenate all DataFrames
df_fifa = pd.concat(fifa, ignore_index=True)

# Save to CSV
df_fifa.to_csv('fifa_worldcup_historical_data.csv', index=False)
#Getting fixture
df_fixture = get_matches(2022 )
df_fixture.to_csv('fifa_worldcup_fixture_data.csv', index = False)


# GATHERING THE MISSING DATA

In [ ]:
# MISSING DATA

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

path =  'C:/Users/Divya/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)


def get_misssing_data(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

    driver.get(web)
    matches = driver.find_elements(by='xpath', value='//td[@align="right"]/.. | //td[@style="text-align:right;"]/..')
    # matches = driver.find_elements(by='xpath', value='//tr[@style="font-size:90%"]')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find_element(by='xpath', value='./td[1]').text)
        score.append(match.find_element(by='xpath', value='./td[2]').text)
        away.append(match.find_element(by='xpath', value='./td[3]').text)

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    time.sleep(2)
    return df_football


years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

fifa = [get_misssing_data(year) for year in years]
driver.quit()
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv("fifa_worldcup_missing_data.csv", index=False)

# DATA CLEANING


In [2]:
df_historical_data = pd.read_csv('fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixtures.csv')
df_missing = pd.read_csv('fifa_worldcup_missing_data.csv')

# Cleaning the fixtutre data

In [3]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

# CLEANING MISSING DATA 

In [4]:
df_missing[df_missing['home'].isnull()]
df_missing.dropna(inplace=True)
df_missing

,home,score,away,year
0,Italy,1–0,Austria,1990
1,United States,1–5,Czechoslovakia,1990
2,Italy,1–0,United States,1990
3,Austria,0–1,Czechoslovakia,1990
4,Italy,2–0,Czechoslovakia,1990
5,Austria,2–1,United States,1990
6,Argentina,0–1,Cameroon,1990
7,Soviet Union,0–2,Romania,1990
8,Argentina,2–0,Soviet Union,1990
9,Cameroon,2–1,Romania,1990


# Cleaning missing data

In [5]:
# Droping the null values
df_missing.dropna(inplace=True)
#concatenat DFs missing and historical data
df_historical_data = pd.concat([df_historical_data,df_missing],ignore_index=True)
df_historical_data.drop_duplicates(inplace = True)
df_historical_data.sort_values('year', inplace = True)
df_historical_data

,home,score,away,year
0,France,4–1,Mexico,1930
13,United States,3–0,Paraguay,1930
14,Paraguay,1–0,Belgium,1930
15,Argentina,6–1,United States,1930
16,Uruguay,6–1,Yugoslavia,1930
...,...,...,...,...
853,Brazil,2–0,Mexico,2018
860,Russia,2–2 (a.e.t.),Croatia,2018
859,Sweden,0–2,England,2018
858,Brazil,1–2,Belgium,2018


# CLEANING HISTORICAL DATAFRAME

In [7]:
#Deleting the walkover match data
delete_index = df_historical_data[df_historical_data['home'].str.contains('Sweden')&
                                   df_historical_data['away'].str.contains('Austria')].index
df_historical_data.drop(index=delete_index, inplace = True)

In [8]:
df_historical_data[df_historical_data['score'].str.contains('[^\\d-]')]
df_historical_data.to_csv('nwx.csv',index=False)

In [12]:
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\\d-]', '-', regex=True)

In [13]:
df_historical_data.to_csv('btr.csv', index = False)

In [14]:
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

In [9]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('btr.csv')

# Convert the column to string and insert hyphens between characters
df['score'] = df['score'].astype(str).apply(lambda x: '-'.join(x))

# Save the modified CSV
df.to_csv('modified_btr.csv', index=False)

In [10]:
df.to_csv('historical_data.csv', index = False)


In [11]:
df_historical_data = pd.read_csv('historical_data.csv')

In [1]:
import pandas as pd

# Load the CSV file (replace with your actual file path)
df_historical_data_1 = pd.read_csv('historical_data.csv')

# Define the regex pattern to match "X-Y" (e.g., 4-5, 10-2)
pattern = r'^\d+-\d+$'

# Keep rows where 'score' matches the pattern and drop others
df_historical_data_1 = df_historical_data_1[df_historical_data_1['score'].str.match(pattern, na=False)]

# Save the cleaned data
df_historical_data_1.to_csv('cleaned_historical_data.csv', index=False)
print("Cleaned data saved to 'cleaned_historical_data.csv'!")

Cleaned data saved to 'cleaned_historical_data.csv'!


In [16]:
df_historical_data_1[['Homegoals','Away']] = df_historical_data_1['score'].str.split('-', expand = True)

In [17]:
df_historical_data_1

,home,score,away,year,Homegoals,Away
0,France,4-1,Mexico,1930,4,1
1,United States,3-0,Paraguay,1930,3,0
2,Paraguay,1-0,Belgium,1930,1,0
3,Argentina,6-1,United States,1930,6,1
4,Argentina,6-3,Mexico,1930,6,3
...,...,...,...,...,...,...
894,Iran,1-1,Portugal,2018,1,1
895,Spain,2-2,Morocco,2018,2,2
896,France,2-1,Australia,2018,2,1
898,Denmark,1-1,Australia,2018,1,1


In [31]:
df_historical_data_1.drop('score',axis=1, inplace=True)

In [32]:
df_historical_data_1

,HomeTeam,AwayTeam,Year,Homegoals,Away,TotalGoals
0,France,Mexico,1930,4,1,5
1,United States,Paraguay,1930,3,0,3
2,Paraguay,Belgium,1930,1,0,1
3,Argentina,United States,1930,6,1,7
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
894,Iran,Portugal,2018,1,1,2
895,Spain,Morocco,2018,2,2,4
896,France,Australia,2018,2,1,3
898,Denmark,Australia,2018,1,1,2


In [33]:
df_historical_data_1.rename(columns={'home': 'HomeTeam','away':'AwayTeam','year':'Year'}, 
                           inplace = True)
df_historical_data_1 = df_historical_data_1.astype({'Homegoals':int, 'Away':int , 'Year':int})

In [34]:
df_historical_data_1.dtypes

HomeTeam      object
AwayTeam      object
Year           int64
Homegoals      int64
Away           int64
TotalGoals     int64
dtype: object

In [35]:
df_historical_data_1['TotalGoals'] = df_historical_data_1['Homegoals']+df_historical_data_1['Away']

In [36]:
df_historical_data_1

,HomeTeam,AwayTeam,Year,Homegoals,Away,TotalGoals
0,France,Mexico,1930,4,1,5
1,United States,Paraguay,1930,3,0,3
2,Paraguay,Belgium,1930,1,0,1
3,Argentina,United States,1930,6,1,7
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
894,Iran,Portugal,2018,1,1,2
895,Spain,Morocco,2018,2,2,4
896,France,Australia,2018,2,1,3
898,Denmark,Australia,2018,1,1,2


In [37]:
df_historical_data_1.to_csv('cleaned_fifa_worldcup_matches.csv', index=False)
df_fixture.to_csv('cleaned_fifa_worldcup_fixtures.csv', index=False)

In [16]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("cleaned_fifa_worldcup_fixtures.csv")

# Replace 'score' column with Match 1-64 values
if 'score' in df.columns:
    df['score'] = [f'Match {i+1}' for i in range(len(df))]
else:
    print("Score column not found in the dataframe")

# Save the modified dataframe (replace with same path to overwrite)
df.to_csv("modified_fixtures.csv", index=False)

In [26]:
pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:
for group in dict_table:
    print(dict_table[group]['Team'].values)

['Argentina' 'Saudi Arabia' 'Mexico' 'Poland']
['Netherlands' 'Senegal' 'Ecuador' 'Qatar (H)']
['England' 'United States' 'Iran' 'Wales']
['Argentina' 'Poland' 'Mexico' 'Saudi Arabia']
['France' 'Australia' 'Tunisia' 'Denmark']
['Japan' 'Spain' 'Germany' 'Costa Rica']
['Morocco' 'Croatia' 'Belgium' 'Canada']
['Brazil' 'Switzerland' 'Cameroon' 'Serbia']


In [2]:
from scipy.stats import poisson

In [55]:
import pandas as pd

# Define expected columns for all groups
expected_columns = ['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']

for group_name, df in dict_table.items():
    # Add missing columns and fill with 0
    missing_cols = [col for col in expected_columns if col not in df.columns]
    for col in missing_cols:
        df[col] = 0
    
    # Reorder columns to match expected format
    df = df[expected_columns]
    
    # Zero numeric columns (everything except Pos/Team)
    numeric_cols = [col for col in expected_columns if col not in ['Pos', 'Team']]
    df[numeric_cols] = 0
    df[numeric_cols] = df[numeric_cols].astype(int)
    
    dict_table[group_name] = df

# Verify Group A now matches others
print("Group A columns after standardization:")
print(dict_table['Group A'].columns.tolist())

print("\nSample Group A data:")
print(dict_table['Group A'].head())

Group A columns after standardization:
['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']

Sample Group A data:
  Pos          Team  Pld  W  D  L  GF  GA  GD  Pts
0  C1     Argentina    0  0  0  0   0   0   0    0
1  C2  Saudi Arabia    0  0  0  0   0   0   0    0
2  C3        Mexico    0  0  0  0   0   0   0    0
3  C4        Poland    0  0  0  0   0   0   0    0


In [19]:
import pandas as pd
import pickle

# Load data
df_historical_data = pd.read_csv('cleaned_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('modified_fixtures.csv')

# Process home data
df_home = df_historical_data[['HomeTeam', 'Homegoals', 'Away']].rename(
    columns={'HomeTeam': 'Team', 'Homegoals': 'GoalsScored', 'Away': 'GoalsConceded'}
)

# Process away data
df_away = df_historical_data[['AwayTeam', 'Away', 'Homegoals']].rename(
    columns={'AwayTeam': 'Team', 'Away': 'GoalsScored', 'Homegoals': 'GoalsConceded'}
)

# Concatenate and group to ensure unique teams
df_Team_strenght = pd.concat([df_home, df_away], ignore_index=True)
df_Team_strenght = df_Team_strenght.groupby('Team').mean().reset_index()

# Clean team names to avoid duplicates
df_Team_strenght['Team'] = df_Team_strenght['Team'].str.strip().str.lower()

# Save to CSV (ensure no index)
df_Team_strenght.to_csv('Tstrenght.csv', index=False)

In [20]:
import pandas as pd
from scipy.stats import poisson

# Read CSV and clean data
df_Team_strenght = pd.read_csv('Tstrenght.csv')
df_Team_strenght['Team'] = df_Team_strenght['Team'].str.strip().str.lower()
df_Team_strenght.set_index('Team', inplace=True)

# Ensure no duplicates in the index
df_Team_strenght = df_Team_strenght[~df_Team_strenght.index.duplicated(keep='first')]

def predict_points(home, away):
    home = home.strip().lower()
    away = away.strip().lower()
    
    if home in df_Team_strenght.index and away in df_Team_strenght.index:
        # Extract scalar values (not Series)
        goals_scored_home = df_Team_strenght.at[home, 'GoalsScored']
        goals_conceded_away = df_Team_strenght.at[away, 'GoalsConceded']
        goals_scored_away = df_Team_strenght.at[away, 'GoalsScored']
        goals_conceded_home = df_Team_strenght.at[home, 'GoalsConceded']
        
        lamb_home = goals_scored_home * goals_conceded_away
        lamb_away = goals_scored_away * goals_conceded_home
        
        prob_home, prob_away, prob_draw = 0.0, 0.0, 0.0
        for x in range(0, 11):
            for y in range(0, 11):
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = round(3 * prob_home + prob_draw, 2)
        points_away = round(3 * prob_away + prob_draw, 2)
        return (points_home, points_away)
    else:
        print(f"Team not found. Available teams: {df_Team_strenght.index.tolist()}")
        return (0, 0)

# Test
home_team = input("Enter home team: ")
away_team = input("Enter away team: ")
home_points, away_points = predict_points(home_team, away_team)
print(f"Expected points for {home_team}: {home_points}")
print(f"Expected points for {away_team}: {away_points}")

Enter home team:  Colombia
Enter away team:  Austria


Expected points for Colombia: 1.79
Expected points for Austria: 0.61


In [58]:
predict_points('Austria', 'Colombia')

(np.float64(0.61), np.float64(1.79))

In [16]:
print("Teams in index:", df_Team_strenght.index.tolist())

Teams in index: [' Austria', ' Colombia', ' Costa Rica', ' Czechoslovakia', ' Egypt', ' Netherlands', ' Republic of Ireland', ' Romania', ' Scotland', ' South Korea', ' Soviet Union', ' Spain', ' Sweden', ' United Arab Emirates', ' United States', ' Uruguay', ' Yugoslavia', 'Algeria\xa0', 'Argentina ', 'Argentina\xa0', 'Australia\xa0', 'Austria ', 'Austria\xa0', 'Belgium ', 'Belgium\xa0', 'Bolivia\xa0', 'Bosnia and Herzegovina\xa0', 'Brazil ', 'Brazil\xa0', 'Bulgaria\xa0', 'Cameroon ', 'Cameroon\xa0', 'Chile\xa0', 'Colombia\xa0', 'Costa Rica ', 'Costa Rica\xa0', 'Croatia\xa0', 'Cuba\xa0', 'Czechoslovakia\xa0', 'Denmark\xa0', 'East Germany\xa0', 'Ecuador\xa0', 'England ', 'England\xa0', 'FR Yugoslavia\xa0', 'France\xa0', 'Germany\xa0', 'Ghana\xa0', 'Greece\xa0', 'Honduras\xa0', 'Hungary\xa0', 'Iceland\xa0', 'Iran\xa0', 'Iraq\xa0', 'Italy ', 'Italy\xa0', 'Ivory Coast\xa0', 'Jamaica\xa0', 'Japan\xa0', 'Mexico\xa0', 'Morocco\xa0', 'Netherlands ', 'Netherlands\xa0', 'New Zealand\xa0', 'Nige

# PREDICTION OF WORLD CUP

# GROUP STAGE MATCHES

In [64]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [65]:
df_fixture_group_48

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 3,Senegal,2022
3,Netherlands,Match 4,Ecuador,2022
4,Ecuador,Match 5,Senegal,2022
5,Netherlands,Match 6,Qatar,2022
6,England,Match 7,Iran,2022
7,United States,Match 8,Wales,2022
8,Wales,Match 9,Iran,2022
9,England,Match 10,United States,2022


In [68]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,C1,Argentina,0,0,0,0,0,0,0,0
1,C2,Saudi Arabia,0,0,0,0,0,0,0,0
2,C3,Mexico,0,0,0,0,0,0,0,0
3,C4,Poland,0,0,0,0,0,0,0,0


In [76]:
# Convert 'Pts' to float first to handle decimal predictions
for group in dict_table:
    dict_table[group]['Pts'] = dict_table[group]['Pts'].astype(float)

# Preprocess team names to lowercase for consistent matching
df_fixture_group_48['home'] = df_fixture_group_48['home'].str.lower().str.strip()
df_fixture_group_48['away'] = df_fixture_group_48['away'].str.lower().str.strip()

for group in dict_table:
    # Get teams in lowercase for matching
    dict_table[group]['Team'] = dict_table[group]['Team'].str.lower().str.strip()
    teams_in_group = dict_table[group]['Team'].values
    
    # Filter fixtures for this group
    mask = df_fixture_group_48['home'].isin(teams_in_group) & \
           df_fixture_group_48['away'].isin(teams_in_group)
    df_fixture_group = df_fixture_group_48[mask]
    
    for index, row in df_fixture_group.iterrows():
        home = row['home'].lower().strip()
        away = row['away'].lower().strip()
        
        # Check if teams exist in group
        if home not in teams_in_group:
            print(f"Warning: {home} not found in {group}")
            continue
        if away not in teams_in_group:
            print(f"Warning: {away} not found in {group}")
            continue
            
        points_home, points_away = predict_points(home, away)
        
        # Update points as floats
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += float(points_home)
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += float(points_away)
    
    # Sort and format final table
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index(drop=True)
    dict_table[group]['Pts'] = dict_table[group]['Pts'].round(0).astype(int)  # Convert to integer points
    dict_table[group]['Team'] = dict_table[group]['Team'].str.title()  # Restore proper casing

# Verify results
print("Processed Group H:")
print(dict_table['Group H'])

Processed Group H:
          Team  Pts
0       Brazil   56
1     Cameroon   40
2  Switzerland   32
3       Serbia    8


# KNOCKOUT

In [84]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout


,home,score,away,year,winner
48,Netherlands,Match 49,United States,2022,?
49,Argentina,Match 50,Australia,2022,?
50,France,Match 51,Poland,2022,?
51,England,Match 52,Senegal,2022,?
52,Japan,Match 53,Croatia,2022,?
53,Brazil,Match 54,South Korea,2022,?
54,Morocco,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [85]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [87]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,United States,2022,Netherlands
49,Argentina,Match 50,Australia,2022,Argentina
50,France,Match 51,Poland,2022,France
51,England,Match 52,Senegal,2022,England
52,Japan,Match 53,Croatia,2022,Croatia
53,Brazil,Match 54,South Korea,2022,Brazil
54,Morocco,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


# QUATER FINALS 

In [88]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [89]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Croatia,Match 57,Brazil,2022,?
57,Netherlands,Match 58,Argentina,2022,?
58,Morocco,Match 59,Portugal,2022,?
59,England,Match 60,France,2022,?


In [90]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Croatia,Match 57,Brazil,2022,Brazil
57,Netherlands,Match 58,Argentina,2022,Argentina
58,Morocco,Match 59,Portugal,2022,Portugal
59,England,Match 60,France,2022,France


# SEMI FINALS

In [91]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Argentina,Match 61,Croatia,2022,?
61,France,Match 62,Morocco,2022,?


In [92]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Argentina,Match 61,Croatia,2022,Argentina
61,France,Match 62,Morocco,2022,France


# FINAL PREDICTION MATCH NO.63

In [95]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Croatia,Match 63,Morocco,2022,?
63,Argentina,Match 64,France,2022,?


In [96]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Croatia,Match 63,Morocco,2022,Croatia
63,Argentina,Match 64,France,2022,Argentina


# FINAL WINNER OF MATCH NO> 63 IS ARGENTINA